In [2]:
!pip install dash

You should consider upgrading via the 'C:\Users\Lenovo\OneDrive\Escritorio\python.exe -m pip install --upgrade pip' command.


In [3]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import dash_table
import plotly.express as px
import psycopg2
import pandas as pd

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_15080\1062133182.py:4: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table


In [143]:

# Conecta a la base de datos PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    database="mav2024",
    user="postgres",
    password="Camila1995."
)

# Define una función para cargar los datos de PostgreSQL en un DataFrame
def carga_df_psql():
    query = "SELECT * FROM mav2024;"
    df = pd.read_sql(query, conn)
    return df

# Carga los datos de PostgreSQL en un DataFrame
df = carga_df_psql()

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_15080\4111266347.py:12: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [7]:
from datetime import datetime, timedelta


In [8]:
#Definimos algunas fechas 
# Obtener la fecha actual
fecha_actual = datetime.now()

# Construir la fecha de inicio del año actual
fecha_inicio_anio = datetime(fecha_actual.year, 1, 1)

# Formato de fecha para la API (DD/MM/AA)
fecha_formato_api = "%d/%m/%y"

In [147]:
df['Mes']=df['Fecha'].dt.month

coltab1=['Tipo Instrumento', ' Monto', 'Mes']
# Define la propiedad 'columns' con las columnas seleccionadas
coltab2=['Segmento', 'Periodo', ' Tasa']

In [40]:
df_tab1 = df[coltab1].groupby(['Tipo Instrumento', 'Mes']).sum().groupby(level=0).cumsum().reset_index()
df_tab2= df[coltab2].groupby(['Segmento', 'Periodo']).mean().groupby(level=0).cumsum().reset_index()


In [145]:
df['Tipo Instrumento'] = df['Tipo Instrumento'].astype(float)  # Convertir la columna a tipo flotante

ValueError: could not convert string to float: 'PAGARE'

In [152]:
pivot_table1 = df.pivot_table(index='Tipo Instrumento', columns='Mes', values=' Monto', aggfunc='sum', fill_value=0).reset_index()
pivot_table1

Mes,Tipo Instrumento,1,2,3,4
0,CPD,6750000,0,0,0
1,ECHEQ,2200440537,1308777724,2310032754,592540522
2,PAGARE,183588,38000,11000,247000


In [153]:
# Lista de nombres de las columnas que deseas incluir en la suma
columnas_a_sumar = pivot_table1.columns[1:]  # Excluye la primera columna

# Suma por fila, saltando la primera columna
pivot_table1['Total'] = pivot_table1[columnas_a_sumar].sum(axis=1)
pivot_table1

Mes,Tipo Instrumento,1,2,3,4,Total
0,CPD,6750000,0,0,0,6750000
1,ECHEQ,2200440537,1308777724,2310032754,592540522,6411791537
2,PAGARE,183588,38000,11000,247000,479588


In [154]:
# Suma por columna
suma_por_columna = pivot_table1.iloc[:, 1:].sum()

# Agregar la fila de suma por columna a pivot_table1
pivot_table1.loc['Total por Columna'] = ['Total'] + suma_por_columna.tolist()
pivot_table1

Mes,Tipo Instrumento,1,2,3,4,Total
0,CPD,6750000,0,0,0,6750000
1,ECHEQ,2200440537,1308777724,2310032754,592540522,6411791537
2,PAGARE,183588,38000,11000,247000,479588
Total por Columna,Total,2207374125,1308815724,2310043754,592787522,6419021125


In [55]:
pivot_table2 = df.pivot_table(index='Segmento', columns='Periodo', values=' Tasa', aggfunc='mean', fill_value=0).reset_index()
pivot_table2

Periodo,Segmento,0 a 60,120 a 180,180 a 210,210 a 240,240 a 300,60 a 90,90 a 120,Más de 300
0,Avalado,69.907643,91.060000,101.875,103.75,0.0,79.060748,92.627273,1.333333
1,DIRECTO,85.000000,4.000000,0.000,0.00,5.0,0.000000,110.000000,7.500000
2,GARANTIZADO,90.838095,104.666667,135.000,135.00,135.0,93.558559,104.625000,135.000000


In [127]:
pivot_table1 = pivot_table1.apply(pd.to_numeric, errors='ignore')


In [141]:
# Agregar totales por fila (totales por tipo de instrumento)
pivot_table1['Total por Instrumento'] = pivot_table1.sum()
pivot_table1


Mes,Tipo Instrumento,1,2,3,4,Total por Instrumento
0,CPD,6.750000e+06,0.000000e+00,0.000000e+00,0.0,NaN
1,ECHEQ,2.200441e+09,1.308778e+09,2.310033e+09,592540522.0,2207374125.0
2,PAGARE,1.835880e+05,3.800000e+04,1.100000e+04,247000.0,1308815724.0


In [142]:
pivot_table1['Total por Instrumento'] = pivot_table1.fillna(0).sum(axis=1)
pivot_table1

TypeError: can only concatenate str (not "float") to str

In [80]:
meses = df['Mes'].unique()


In [170]:
##Acumulado para barras
accumulated_df= df.groupby(['Mes', 'Segmento'])[' Monto'].sum().reset_index()
accumulated_df

,Mes,Segmento,Monto
0,1,Avalado,1184679557
1,1,DIRECTO,20082000
2,1,GARANTIZADO,1002612568
3,2,Avalado,849805724
4,2,DIRECTO,12010000
5,2,GARANTIZADO,447000000
6,3,Avalado,959043154
7,3,DIRECTO,11000
8,3,GARANTIZADO,1350989600
9,4,Avalado,530787522


In [171]:


# Supongamos que ya tienes definido tu dataframe df

app = dash.Dash(__name__)

# Define tus datos, por ejemplo df para el dataframe de tus datos

app.layout = html.Div([
        html.H1("MAV 2024 ARG VALORES"),
    
        html.Label("Selecciona un mes:"),
        dcc.Dropdown(
        id='month-dropdown',
        options=[{'label': mes, 'value': mes} for mes in meses],
        value=meses.tolist(),
        multi=True  # Establecer el valor predeterminado como el primer mes en el dataframe
    ),
        
    html.Label("Selecciona Segmento:"),
    dcc.Dropdown(
        id='category-dropdown',
        options=[{'label': category, 'value': category} for category in df['Segmento'].unique()],  # Define las opciones del dropdown
        value=[],
        multi=True,
    ),
    html.Div([
        dash_table.DataTable(
            id='table-1',
            columns=[{'name': str(col), 'id': str(col)} for col in pivot_table1.columns],  # Define las columnas de la tabla
            data=pivot_table1.to_dict('records'),
            style_table={'overflowX': 'auto'},  # Ajustar el ancho de la tabla al contenido
            style_cell={'textAlign': 'center'},  # Define los datos de la tabla
        )
    ], style={'display': 'inline-block', 'width': '48%','marginRight': '2%'}),
    html.Div([
        dash_table.DataTable(
            id='table-2',
            columns=[{'name': str(col), 'id': str(col)} for col in pivot_table2.columns],  
            data=pivot_table2.to_dict('records'),  
            style_table={'overflowX': 'auto'},  # Ajustar el ancho de la tabla al contenido
            style_cell={'textAlign': 'center'},
        )
    ], style={'display': 'inline-block', 'width': '50%'}),
    html.Div([
        dcc.Graph(
            id='scatter-plot',
            figure=px.scatter(df, x='dias_entre_fechas', y=' Tasa', color='Segmento', title='Scatter Plot'),  # Define la figura del gráfico de dispersión
        )
    ], style={'display': 'inline-block', 'width': '50%'}),
    html.Div([
        dcc.Graph(
            id='bar-chart',
            figure=px.bar(accumulated_df, x='Mes', y=' Monto', color='Segmento', barmode='stack', title='Insturmentos operados por mes'),  # Define la figura del gráfico de barras
        )
    ], style={'display': 'inline-block', 'width': '50%'})
    
    html.A("Enlace a Google", href="https://www.google.com", target="_blank")

])

# Callback para actualizar las tablas


# Callback para actualizar las tablas
# Callback para actualizar las tablas
@app.callback(
    Output('table-1', 'data'),
    Output('table-2', 'data'),
    Output('scatter-plot', 'figure'),
    Output('bar-chart', 'figure'),
    Input('month-dropdown', 'value'),
    Input('category-dropdown', 'value')
)

def update_tables(selected_month, categories):
    # Copiar el DataFrame original para evitar modificarlo directamente
    filtered_df = df.copy()

    # Filtrar por mes seleccionado
    if selected_month:  
        filtered_df = filtered_df[filtered_df['Mes'].isin(selected_month)]

    # Filtrar por categorías seleccionadas
    if categories:
        filtered_df = filtered_df[filtered_df['Segmento'].isin(categories)]
    
    # Crear tablas pivotantes y figuras
    # Suma por fila, saltando la primera columna
    pivot_table1['Total'] = pivot_table1[columnas_a_sumar].sum(axis=1)
    pivot_table1.loc['Total por Columna'] = ['Total'] + suma_por_columna.tolist()

    pivot_table2 = filtered_df.pivot_table(index='Segmento', columns='Periodo', values=' Tasa', aggfunc='mean', fill_value=0).reset_index()
    scatter_fig = px.scatter(filtered_df, x='dias_entre_fechas', y=' Tasa', color='Segmento', title='Scatter Plot')
    bar_fig =  px.bar(accumulated_df, x='Mes', y=' Monto', color='Segmento', barmode='stack', title='Instrumentos operados por mes')
    pivot_table2 = pivot_table2.round(2)

    # Devolver los resultados para los componentes de la interfaz de usuario
    return pivot_table1.to_dict('records'), pivot_table2.to_dict('records'), scatter_fig, bar_fig
if __name__ == '__main__':
    app.run_server(debug=True)
